<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_%EC%9D%B4%EC%86%8C%ED%98%84_%EC%84%9C%EB%B9%84%EC%8A%A42_%EB%AA%A8%EB%8D%B8%EB%A7%812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00


In [3]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ad612fc2fcf8ca6779000c9a52c44d43a703505582bac7ffcd162c0077c05c11
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch
import torch.nn as nn
from torch import optim
from torch import optim

import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [7]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 데이터 불러오기

In [8]:
pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct.csv").head(3)

,review,reply,star,star_t,star_q,star_d,food,store,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,5,5.0,5.0,5.0,직화 삼겹옛날도시락,['1988응답하라추억의옛날도시락 방이점'],1
1,기름 있는 부분이 생각보다 많아요 참고하세요,소중한 의견 남겨 주셔서 감사합니다 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 노...,5,5.0,5.0,5.0,직화 삼겹옛날도시락 볶음김치,['1988응답하라추억의옛날도시락 방이점'],0
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,5.0,5.0,5.0,직화 삼겹옛날도시락 우삼겹청국장찌개 대접밥,['1988응답하라추억의옛날도시락 방이점'],1


In [9]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct.csv")
cols = ['review','reply','sentiment']
train = train[cols]
train.isnull().sum()

review        0
reply        48
sentiment     0
dtype: int64

In [10]:
idx_null = train[train['reply'].isnull()].index
train = train.drop(idx_null, axis=0)
train.isnull().sum()

review       0
reply        0
sentiment    0
dtype: int64

In [11]:
train['sentiment'].value_counts().sort_index() # 긍부정 분포 확인

0     1813
1    35279
Name: sentiment, dtype: int64

In [12]:
print(len((train[train['sentiment'] == 0])))
len(train[train['sentiment'] == 0])/len(train[train['sentiment'] == 1])

1813


0.051390345531335924

# 부정데이터 삭제

In [13]:
train = train[train['sentiment'] == 1]
train

,review,reply,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,1
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1
3,,저희 매장과 함께 편안한 시간 되셨을까요 1 고객을 생각하는 마음 2 더 나은 맛과...,1
4,냉면을 따로 시키고 미니 냉면을 합친 것인데 양이 좀 적어 보이고 김치 정식도 양이...,주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요 고객님의 행복한 시간에 저희...,1
5,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께해 주셔서 영광 다음에도 고객님의 즐거운 순간을 함...,1
...,...,...,...
37135,맛있어요 저번에 주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉...,고객님 안녕하세요 소중한 리뷰 너무 감사합니다 재주문해 주셨는데 국물 양이 살짝 아...,1
37136,맛있어요 또 시킬게요,고객님 안녕하오 소중한 리뷰 남겨주셔 서 정말 달 감사합니다 만족스러운 식사가 되셨...,1
37137,정말 맛있게 잘 먹었어요 자주 시켜 먹을게요,고객님 안녕하세요 맛있게 드셨다는 말씀에 힘이 나는 거 같습니다 항상 변함없는 맛으...,1
37138,와우 여기 뭐죠 주변에서 맛있다고 먹어보라 해서 시켰는데 육수가 일단 시원 칼칼하니...,고객님 안녕하세요 맛있게 드셔주시고 정성스러운 사진 리뷰까지 정말 감사합니다 매일매...,1


### 오버샘플링(긍정->부정 데이터)

In [14]:
pd.read_csv(f"{DATA_PATH}train_gpt_9300.csv").head(3)

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0


In [15]:
train_gpt = pd.read_csv(f"{DATA_PATH}train_gpt_9300.csv")
train_gpt.isnull().sum()

gpt_review    0
gpt_reply     0
sentiment     0
dtype: int64

In [16]:
# 컬럼명 통일
rename_col = {'gpt_review': 'review', 'gpt_reply': 'reply'}
train_gpt.rename(columns=rename_col, inplace=True)
train_gpt

,review,reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
9318,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0
9319,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0
9320,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0
9321,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0


In [17]:
# 합치기
train = pd.concat([train, train_gpt], axis=0).reset_index(drop=True)
train

,review,reply,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,1
1,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1
2,,저희 매장과 함께 편안한 시간 되셨을까요 1 고객을 생각하는 마음 2 더 나은 맛과...,1
3,냉면을 따로 시키고 미니 냉면을 합친 것인데 양이 좀 적어 보이고 김치 정식도 양이...,주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요 고객님의 행복한 시간에 저희...,1
4,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께해 주셔서 영광 다음에도 고객님의 즐거운 순간을 함...,1
...,...,...,...
44597,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0
44598,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0
44599,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0
44600,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0


In [18]:
train['review'] = train['review'].astype(str)
train['reply'] = train['reply'].astype(str)

### 여기까지 저장하기

In [ ]:
# train.to_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct_gpt.csv", index=False)

# 여기까지 완료된 데이터 불러오기

In [ ]:
# train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908_clean_remove_posneg_correct_gpt.csv")
# train.head(3)

,review,reply,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,1
1,기름 있는 부분이 생각보다 많아요 참고하세요,소중한 의견 남겨 주셔서 감사합니다 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 노...,0
2,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1


In [19]:
train.isnull().sum()

review       0
reply        0
sentiment    0
dtype: int64

## 추가 전처리

### 개별 삭제(추론 결과보고 계속 추가)

In [20]:
print(train[train['reply'].str.contains("빕스")].index)

Int64Index([43621], dtype='int64')


In [21]:
train['reply'][43621]

'안녕하세요. 불만족스러운 경험을 하신 것 같아 정말 죄송합니다. 빕스에서 제공하는 음식에 대해 항상 고객님의 요구에 부응할 수 있도록 노력하고 있습니다. 고객님께서는 맛에 대해서도 불만족스러움을 느끼셨다고 하셨는데, 다음에는 더 면밀히 조사하여 개선하는 데 노력하도록 하겠습니다. 케이크도 맛에 불만족스러우셨다니 정말 죄송합니다. 저희는 언제나 최상의 맛과 서비스를 제공하고자 하고 있습니다. 더 나은 경험을 제공할 수 있도록 노력하겠습니다. 감사합니다. '

In [22]:
remove_keywords = ['직구 쌈','직구 삼','빕스','계림','부잣집']
for keyword in remove_keywords:
    train['reply'] = train['reply'].str.replace(keyword, '')

train['reply'][43621]

'안녕하세요. 불만족스러운 경험을 하신 것 같아 정말 죄송합니다. 에서 제공하는 음식에 대해 항상 고객님의 요구에 부응할 수 있도록 노력하고 있습니다. 고객님께서는 맛에 대해서도 불만족스러움을 느끼셨다고 하셨는데, 다음에는 더 면밀히 조사하여 개선하는 데 노력하도록 하겠습니다. 케이크도 맛에 불만족스러우셨다니 정말 죄송합니다. 저희는 언제나 최상의 맛과 서비스를 제공하고자 하고 있습니다. 더 나은 경험을 제공할 수 있도록 노력하겠습니다. 감사합니다. '

### 단문 데이터 제거
- 데이터 정제 과정에서 무의미해진 데이터는 학습데이터로 사용불가
    - 리뷰 : 공백만 생겼거나, 리뷰 이벤트를 위한 단답형 제거
    - 답변 : 16자 이하 답변 제거

In [23]:
len(train['review'][3]) , len(train['reply'][642])

(66, 47)

In [24]:
train['review'][3] , train['reply'][642]

('냉면을 따로 시키고 미니 냉면을 합친 것인데 양이 좀 적어 보이고 김치 정식도 양이 적어 보이는 거 말고는 만족합니다 ',
 '가성비로 쭉 밀고 나가고 있습니다만 맛도 많이 신경 쓰고 있습니다 쭉쭉 놀러와 주세요')

In [25]:
train[(train['review'].str.len() > 4) & (train['review'].str.len() <= 10)] # 리뷰

,review,reply,sentiment
22,언제나 맛있습니다,소중한 시간에 저희 매장을 다시 이용해 주셔서 감사합니다 만족스러우셨던 거 같아 다...,1
24,잘 먹었습니다,좋은 리뷰를 남겨주셔서 감사해요 저희 팀은 여러분이 웃을 수 있게 항상 최선을 다하...,1
27,양도 많고 맛있어요,잊지 않으시고 리뷰 남겨주셔서 정말 감사합니다 맛있는 메뉴들은 양껏 먹어도 먹고 싶...,1
37,맛있게 잘 먹었어요,이렇게 고객님을 뵙게 되어 너무 영광 준비한 저희 메뉴가 입맛에는 맞으실까 항상 걱...,1
50,양 많고 맛있었어요,저희 매장 찾아주신 것도 감사한데 리뷰까지 감동 고객님들의 배고픔을 저희는 지켜만 ...,1
...,...,...,...
41130,너무 달았어요,단맛이 강한 음식이 었습니다. 앞으로는 맛을 조절하여 제공하도록 노력하겠습니다.,0
41287,맛이 별로였어요,"죄송해요, 많이 실망하셨군요. 저희 음식이 맛있게 전달되도록 조금 더 노력하겠습니다.",0
42357,맛이 별로였어요,"죄송합니다, 맛에 대해 별점을 받으셨다니 정말 안타깝습니다. 더 맛있는 음식으로 보...",0
42647,맛이 별로였어요.,죄송합니다. 맛이 별로였다니 정말 안타깝네요. 다음에는 더 좋은 맛으로 보답하겠습니다.,0


In [26]:
train[(train['reply'].str.len() > 15) & (train['reply'].str.len() <= 17)] # 답변

,review,reply,sentiment
518,맛있어요 감사 허무니 당,네넵 감사 감사 감사 허무니 당,1
520,배달도 예상시간보다 빨리 왔고 치즈도 엄청 많이 왔어요 진짜 맛있습니다,양도 맛도 항상 진심 감사합니다,1
600,오오 떡볶이도 진짜 맛있고 튀김도 맛있네요,감사합니다 자주 이용해 주세요,1
629,아쉽게 사진을 찍진 못했지만 맛있게 잘 먹었어요,맛있게 드셨다니 너무 감사합니다,1
632,양도 푸짐하고 맛도 괜찮네요,감사해요 또 만났으면 좋겠습니다,1
...,...,...,...
33123,맛있게 잘 먹었어요,님 맛있게 드셔주셔서 감사합니다,1
34555,맛있었어요 커피도 안 흐르고 잘 왓어옹,맛있게 드셨다니 제가 감사합니다,1
36076,숙주가 많아서 싫어요. 맛도 별로였어요.,불편을 드려서 정말 죄송합니다.,0
36767,맛이 별로였어요. 육수도 많이 없고 갈비도 말라서 실망했어요.,불편을 드려 정말 죄송합니다.,0


In [27]:
len(train[train['review'].str.len() <= 9]) , len(train[train['reply'].str.len() <= 17])

(2570, 923)

In [28]:
train = train[(train['review'].str.len() > 9) & (train['reply'].str.len() > 17)].reset_index(drop=True)
train

,review,reply,sentiment
0,맛있게 잘 먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,1
1,양도 푸짐하고 맛있습니다 청국장 냄새 많이 안 나고 콩도 많이 들어있어서 좋았습니다,늘 최고의 메뉴를 제공해 드리고 싶습니다 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,1
2,냉면을 따로 시키고 미니 냉면을 합친 것인데 양이 좀 적어 보이고 김치 정식도 양이...,주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요 고객님의 행복한 시간에 저희...,1
3,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께해 주셔서 영광 다음에도 고객님의 즐거운 순간을 함...,1
4,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다,안녕하세요 주문해 주셔서 진심으로 감사드려요 이렇게 좋은 리뷰 써 주시니 메뉴 제공...,1
...,...,...,...
41202,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0
41203,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0
41204,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0
41205,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0


## 긍부정 데이터 비율 선택

In [29]:
train['sentiment'].value_counts().sort_index() # 긍부정 분포 확인

0     9205
1    32002
Name: sentiment, dtype: int64

In [30]:
train_neg = train[train['sentiment'] == 0]

# 긍정 데이터 중 중복된 답변제거
train_pos = train[train['sentiment'] == 1].drop_duplicates(subset='reply', keep=False)
train_pos['sentiment'].value_counts().sort_index()

1    14903
Name: sentiment, dtype: int64

In [31]:
length = len(train_neg)
length

9205

In [32]:
# 긍정 데이터를 랜덤으로 추출하기 위해 섞기
shuffled_train_pos = train_pos.sample(frac=1, random_state=42)
train_pos = shuffled_train_pos.iloc[:length].reset_index(drop=True) # 1:1 비율
# train_pos = shuffled_train_pos.iloc[:length*2].reset_index(drop=True) # 2:1 비율 만들기

# 데이터 합치기
train = pd.concat([train_neg, train_pos], ignore_index=True)
train['sentiment'].value_counts().sort_index()

0    9205
1    9205
Name: sentiment, dtype: int64

In [33]:
# 다시 섞기
train = train.sample(frac=1, random_state=42).reset_index(drop=True)
train

,review,reply,sentiment
0,"맛이 별로였어요, 조리 상태도 별로였고 배달 시간도 오래 걸렸네요","안녕하세요~! 맛이 별로였다고 하셨는데, 제 기준에는 싱겁고 딱히 맛있어 보이지 않...",0
1,양도 많고 맛나요,하핫 정말 이런 이런 칭찬에 춤추고 싶네요 든든한 생각나실 때 또 만나요 친절한 고...,1
2,음식의 퀄리티가 낮아서 별로였어요. 맛이 없어서 실망했어요,음식의 퀄리티가 낮아서 실망하셨다니 정말 죄송합니다. 더 맛있는 음식으로 만족스러운...,0
3,리뷰가 좋아서 주문했는데 정말 맛있게 잘 먹었습니다 다 맛있어요 김밥 완전 빅 사이...,감사합니다 언제나 푸짐한 양과 맛있는 맛으로 찾아뵙겠습니다 자주 애용해 주세요,1
4,항상 맛있게 잘 먹고 있습니다,고객님 소중한 고객님이 이임 항상 찾아주시고 맛있게 드셔주시니 너어어어무 기쁘고 행...,1
...,...,...,...
18405,친구시켜줬는데 사진은 이렇지만 고기 많고 맛있대요 감사합니다,저희 매장을 빛내 주셔서 영광 맛깔나는 사진 너무 감사해요 맛있게 드셔 주시기만 해...,1
18406,맛있게 먹는 단골집,자주자주 찾아주세요 저희 이용해 주셔서 감사합니다 최고보다는 최선을 다하는 될 수 ...,1
18407,가격이 좀 비싸서 아쉬웠어요. 좀 더 저렴하면 좋겠어요,가격이 비싸셨다니 정말 죄송합니다. 고객님의 의견을 반영하여 가격을 조정할 수 있도...,0
18408,센스가 없는 배달이었어요.,다음에는 더욱더 만족스러운 서비스를 제공하기 위해 노력하겠습니다.,0


# 모델 불러오기

In [34]:
model_name = "skt/kogpt2-base-v2"

In [35]:
# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [36]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [37]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

# 데이터셋 만들기

In [38]:
train.head()

,review,reply,sentiment
0,"맛이 별로였어요, 조리 상태도 별로였고 배달 시간도 오래 걸렸네요","안녕하세요~! 맛이 별로였다고 하셨는데, 제 기준에는 싱겁고 딱히 맛있어 보이지 않...",0
1,양도 많고 맛나요,하핫 정말 이런 이런 칭찬에 춤추고 싶네요 든든한 생각나실 때 또 만나요 친절한 고...,1
2,음식의 퀄리티가 낮아서 별로였어요. 맛이 없어서 실망했어요,음식의 퀄리티가 낮아서 실망하셨다니 정말 죄송합니다. 더 맛있는 음식으로 만족스러운...,0
3,리뷰가 좋아서 주문했는데 정말 맛있게 잘 먹었습니다 다 맛있어요 김밥 완전 빅 사이...,감사합니다 언제나 푸짐한 양과 맛있는 맛으로 찾아뵙겠습니다 자주 애용해 주세요,1
4,항상 맛있게 잘 먹고 있습니다,고객님 소중한 고객님이 이임 항상 찾아주시고 맛있게 드셔주시니 너어어어무 기쁘고 행...,1


In [39]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        review = item["review"]
        answer = item["reply"]
        sentiment = item["sentiment"]

        return {
            "review": review,
            "answer": answer,
            "sentiment": sentiment
            }

In [40]:
def collate_fn(batch):
    sentiment = torch.tensor([item["sentiment"] for item in batch])
    texts = [f"<q>{item['review']}</s><a>{item['answer']}</s>" for item in batch]
    x = tokenizer(texts, return_tensors="pt", padding=True)

    return {"x": x, "sentiment": sentiment}

In [41]:
dt = ChatDataset(train)
dt[0]

{'review': '맛이 별로였어요, 조리 상태도 별로였고 배달 시간도 오래 걸렸네요 ',
 'answer': '안녕하세요~! 맛이 별로였다고 하셨는데, 제 기준에는 싱겁고 딱히 맛있어 보이지 않았다고 하셨네요. 저희 메뉴의 맛에 대한 평가는 개인의 취향에 따라 달라질 수 있습니다만, 여러분의 의견에 감사드리며 앞으로 더욱 개선해 나가겠습니다! ',
 'sentiment': 0}

In [42]:
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   455,   405, 23373, 15126,  8041,  8006, 13704, 13676, 10008,
           7235, 15126, 19570, 28005, 10135,  7235, 10714,  9539,  7422,  7098,
           8084,   739,     1,  9724,   439,   405,  7967,  7114,  8702,  7801,
           8084,   468,   376, 15882, 15126, 19132,  9078,  7816,  9287,  9037,
          10081,  9052, 16565, 36539, 16317,  8811, 11355, 49067, 14233, 30160,
           9078,  7816,  7098, 25856,  9265,  8806,  9803,  7155,  8143, 11355,
           8022,  9167, 41896, 12695, 46164,  9161, 27852,  9025,  9019, 16913,
           7182, 23636,  9311, 11926, 11955,  8022, 15940, 12854,  7530, 12634,
           9947, 12003,  8711, 12312,  6872, 16913,  7182,   376,   739,     1],
         [ 9724,   455,   405, 28922, 15394, 11355,  7055,  8084,   739,     1,
           9724,   439,   405,  8702,  8709, 29205, 10165, 10165, 19814,  8022,
          38725,  6889, 13358,  7098,  8084, 14309,  7283,  8704,  9658,  7055,
           7892,  90

In [43]:
model(**batch["x"]).logits.shape

torch.Size([2, 90, 51200])

In [44]:
batch["sentiment"].to(device, dtype=torch.float32)

tensor([0., 1.], device='cuda:0')

# 동결

In [45]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [46]:
for name,param in model.named_parameters():
    print(name,param)
    break

transformer.wte.weight Parameter containing:
tensor([[ 0.0282, -0.0369, -0.0061,  ..., -0.0248, -0.0068,  0.0108],
        [ 0.0104,  0.0134,  0.0427,  ...,  0.0091, -0.0053, -0.0012],
        [ 0.0314, -0.0164,  0.0178,  ..., -0.0054, -0.0480,  0.0040],
        ...,
        [-0.0244, -0.0089, -0.0003,  ...,  0.0009,  0.0304, -0.0017],
        [-0.0206,  0.0045,  0.0241,  ...,  0.0630, -0.0192,  0.0099],
        [-0.0199, -0.0382,  0.0268,  ...,  0.0299, -0.0154, -0.0423]],
       requires_grad=True)


## 일부 동결

In [47]:
for name,param in model.named_parameters():
    if not name.startswith("lm_head"):
        param.requires_grad = False
        if name.startswith("transformer.h.9"): # 3/4 동결
            break

In [48]:
for name,param in model.named_parameters():
    print(name,param.requires_grad) # lm_head 만 가중치 업데이트

transformer.wte.weight False
transformer.wpe.weight False
transformer.h.0.ln_1.weight False
transformer.h.0.ln_1.bias False
transformer.h.0.attn.c_attn.weight False
transformer.h.0.attn.c_attn.bias False
transformer.h.0.attn.c_proj.weight False
transformer.h.0.attn.c_proj.bias False
transformer.h.0.ln_2.weight False
transformer.h.0.ln_2.bias False
transformer.h.0.mlp.c_fc.weight False
transformer.h.0.mlp.c_fc.bias False
transformer.h.0.mlp.c_proj.weight False
transformer.h.0.mlp.c_proj.bias False
transformer.h.1.ln_1.weight False
transformer.h.1.ln_1.bias False
transformer.h.1.attn.c_attn.weight False
transformer.h.1.attn.c_attn.bias False
transformer.h.1.attn.c_proj.weight False
transformer.h.1.attn.c_proj.bias False
transformer.h.1.ln_2.weight False
transformer.h.1.ln_2.bias False
transformer.h.1.mlp.c_fc.weight False
transformer.h.1.mlp.c_fc.bias False
transformer.h.1.mlp.c_proj.weight False
transformer.h.1.mlp.c_proj.bias False
transformer.h.2.ln_1.weight False
transformer.h.2.ln_1

## 전체 동결

In [ ]:
# class FreezeNet(torch.nn.Module):
#     def __init__(self, model):
#         super().__init__()
#         self.model = model
#         for param in self.model.parameters():
#             param.requires_grad = False
#         self.model.lm_head = torch.nn.Linear(768,51200)

#     def forward(self,x):
#         return self.model(x)

In [ ]:
# model = FreezeNet(model)
# for param in model.parameters():
#     print(param.requires_grad)

# 학습루프

In [50]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):

        x = batch["x"].to(device)
        sentiment = batch["sentiment"].to(device)

        pred = model(**x).logits  # batch, seq, 단어별 실수값
        n_class = pred.shape[-1]  # 클래스 개수
        sentiment_pred = pred[:, -1, :]

        pred = pred[:, :-1, :]  # eos 토큰 제외
        pred = pred.reshape(-1, n_class)  # 2차원 형태로 변환

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()  # 1차원 벡터 형태로 바뀜

        mask = tgt != tokenizer.pad_token_id  # 패딩토큰 제외 -> 손실을 계산할 때 제외
        tgt = tgt[mask]
        pred = pred[mask]

        loss = loss_fn(pred, tgt)

        sentiment_loss = loss_fn(sentiment_pred, sentiment)

        optimizer.zero_grad()
        total_loss = loss + sentiment_loss
        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

# 하이퍼파라미터 정의

In [ ]:
reset_seeds(SEED)
batch_size = 2
epochs = 10

model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# 일부 동결 #
for name,param in model.named_parameters():
    if not name.startswith("lm_head"):
        param.requires_grad = False
        if name.startswith("transformer.h.3"): # 1/4 동결
            break

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

train_dt = ChatDataset(train)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(f"{i+1}번째 epoch: {train_loss:.4f}")


  0%|          | 0/9205 [00:00<?, ?it/s]

- 0920_nsw_1 : 부정 데이터 only GPT, 1:1 비율. 3/4 동결
- 0920_nsw_2 : 부정 데이터 only GPT, 1:1 비율. 2/4 동결
- 0920_nsw_3 : 부정 데이터 only GPT, 1:1 비율. 1/4 동결
- 0920_nsw_4 : 부정 데이터 only GPT, 1:1 비율. 동결 x

# 학습모델 저장

In [56]:
model.save_pretrained(f"{DATA_PATH}0920_nsw_3")

# 저장한 모델 로드

In [57]:
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0920_nsw_3").to(device)

## 테스트루프

In [60]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask') # 단순 인코딩 기준 가져오기

def evaluate_similarity(input_text, generated_text, alpha=0.9): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [61]:
evaluate_similarity("맛은 있는데 양이 적어요",'맛있게 드셔주셔서 감사합니다. 다음에 양 많이 드릴게요')

0.35823020339012146

In [ ]:
# 옵션
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=512, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=2,
                          no_repeat_ngram_size=4,
                          top_k=50,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

In [62]:
def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=2.0,
                                        num_beams=2,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=4,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=0.8,
                                        top_k=50,
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            # generated_text = paraphrase_text(generated_text)
            generated_text = re.sub(r'[^가-힣]', ' ', generated_text)

            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)
        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [ ]:
# 0920_freeze_1/4
chatbot_with_evaluation(loaded_model,tokenizer,64,device)

In [63]:
# 0920_freeze_2/4
chatbot_with_evaluation(loaded_model,tokenizer,64,device)

user > 맛은 있는데 양이 적어요
(0.48057256937026976, '맛이 나는 것 같네요  먹스타그램  맛스타그램                                       소통  좋')
(0.4591046512126923, '맛이 안나오시죠            마지막으로 저는 마무리를 잘했네요  먹스타그램  맛스타그램                    너')
(0.5120757043361664, '맛이 나는 것 같네요   먹스타그램  맛스타그램                                                     ')
(0.4275734066963196, '맛은 안나오는데         빵순이  먹스타그램  맛스타그램                                        ')
(0.46739691495895386, '맛이 강해서요  그런데 이게 맛있다고 해서       먹스타그램  맛스타그램                                   ')

Best_reply > 맛이 나는 것 같네요   먹스타그램  맛스타그램                                                     
Best_similarity_score: 0.5120757043361664
--------------------------------------------------------------------------------------------
user > quit


In [ ]:
# 0908_gpt_freeze
chatbot_with_evaluation(loaded_model,tokenizer,128,device)

user > 맛은 있는데 양이 적어요
(0.2597696900367737, '감사합니다 저희 이용해 주셔서 감사합니다 최고보다는 최선을 다하는 될 수 있도록 노력하겠습니다 감사합니다와')
(0.3239448130130768, '고객님 맛있게 드셔주셔서 감사합니다 말씀해 주신 내용 참고해서 만족스러운 식사가 되실 수 있게 노력하겠습니다 솔직한 리뷰 감사드립니다')
(0.40282133887795846, '고객님 맛있게 드셔주셔서 감사합니다 항상 정량대로 조리하고 있는데 양이 부족하셨나 봅니다 항상 더 넉넉하게 조리하도록 하겠습니다')
(0.32219595909118653, '맛있게 드셔주셔서 감사합니다 항상 푸짐하게 드리기 위해 노력하는데 만족스러운 식사하신 것 같아 다행이에요 항상 노력하는 되겠습니다 소중한 리뷰 감사드립니다')
(0.2984432280063629, '고객님 안녕하세요 시간 내어 리뷰 작성해 주셔서 정말 감사합니다 항상 고객님들 주문 하나하나에 정성을 담아 대접하겠습니다 믿고 드실 수 있는 기억해 주세요 감사드립니다 찜 댓글 무료 고기      추가해드리고 있습니다 많이 이용해 주세요')
(0.4186217109362284, '고객님 맛있게 드셔주셔서 감사합니다 항상 푸짐하게 드리기 위해 노력하는데 양이 부족하셨다니 죄송합니다 담에 주문하실 때는 더 많이 드릴 수 있도록 하겠습니다')
(0.20933273434638977, '이용해 주셔서 감사합니다 항상 푸짐하게 드실 수 있도록 최선을 다하겠습니다')
(0.33062775135040284, '맛있게 드셔주셔서 감사합니다 항상 푸짐하게 드리기 위해 노력하는데 만족스러운 식사하신 것 같아 다행이에요 다음에 또 생각 나시면 주문해 주세요')
(0.23024382591247558, '소중한 리뷰 감사합니다 맛있게 드셔서 고맙습니다 더 노력하는 닭발이 되겠습니다')
(0.2989537060260773, '고객님 안녕하세요 시간 내어 리뷰 작성해 주셔서 정말 감사합니다 항상 고객님들 주문 하나하나에 정성

- 0914_trainsentiment_0913+gpt

In [ ]:
chatbot(loaded_model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님의 의견은 저희에게 소중한 자산입니다. 계속해서 피드백을 기다리고 있겠습니다. 좀 더 나은 서비스를 제공할 수 있는 방법을 찾아보도록 하겠습니다,, 
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요
bot > 고객님께서 불편을 느끼셨다니 죄송합니다. 좀 더 신속한 배달로 적절한 조치를 취하도록 하겠습니다., 
--------------------------------------------------------------------------------------------
user > 먹다가 머리카락이 나왔어요
bot > 맛있게 드셨다니 감사합니다! 언제나 최선을 다하여 최고의 음식을 드리는 땡초가 되겠습니다, 고객님의 소중한 리뷰에 항상 귀기울이는 벼락으로 보답드릴게요, 자주 애용해주세용~ 신메뉴 혹시라도 배달 받으신다면 잊지 않고 부탁드려요. 적극 반영하겠습니다. 이번주는 급격히 발전하는 한주입니다. 폭염 속 건강 조심하시고 행복하십쇼입니다. 저희 메뉴가 많이 있으니 이용바랍니당 다음에 또 찾아 주실거죠? 감사합니다. 
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉬워요
bot > 고객님의 의견을 반영하여 더 나은 제품을 만들기 위해 노력하겠습니다. 불편을 드려 죄송합니다., 
--------------------------------------------------------------------------------------------
user > 여기 피자가 우리 동네에서 제일 맛있어요,, 쿠폰도 자주 줘서 너무 좋아요
bot > 고객님 안녕하세요. 이렇게 저희에게 귀하고 소중한 평가

- 0905_kogpt2_chat

In [ ]:
chatbot(loaded_model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님 ~! 먼저 죄송합니다. 부족한 부분으로 실망시켜  사과드리며 늘 저희를 사랑해 주시는 고객님이시군요 정성스러운 사진 리뷰와도 너무 감사한데, 먹음직스럽게 찍어주신 소중한 말씀까지.
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 소중한 리뷰 감사드립니다 저희 매장은 맛과 양, 서비스 등 모든 면에서 최고가 되기 위해 항상 최선을 다하고 있습니다, 언제든지 부족함 없이 드실 수 있도록 준비해 보내드리겠습니다.! 앞으로도 만족만 드릴 만반의 준비를 하여 계속 마음속에 남아 있고 싶거든요 <s>~ 좋은 하루 되세요. 고객님 ~
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 오늘 하루를 시작하며 고객님의 리뷰을 보고 웃음 짓게 됩니다 다음에도 점심 식사를 기분 좋게 드실 수 있길 바랍니다. 저희는 항상 맛있는 음식 제공에 힘내며 언제라도 생각나는 맛집이 되도록 노력하겠습니다, 앞으로도 더욱 만족스러운 식사하실 땐 더 큰 기쁨으로 인사드리겠습니다.
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot > 맛있게 드셨다니 다행입니다! 찾아주시는 피드백은 언제든 잘 수렴하고 이를 통해 나날이 발전하는 곳이 되도록 하겠습니다, 더욱더 좋은 모습으로 찾아가겠습니다.<s>. 기쁨 가득한 하루 보내세요 <s>, 감사해요.<s><s><s><s><s> 소중한 리뷰에 제가 느낀 이 행복만큼 고객님도 항상 노력하시길 바라며 오늘

- 0906_negative_kogpt2_chat

In [ ]:
chatbot(model,tokenizer,128,device)

user > 맛이 없어요
bot > 안녕하세요! 저희 맛집이 만족한 식사시간이 되지 못해서 죄송합니다.. 양이 부족하다.?
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 고객님 식사에 불편을 드려 너무 죄송합니다 다음엔 더욱 신경 써서 만족하실 수 있도록 전달하겠습니다, 감사합니다.<s> <unk>얌꿍이 약간 매콤한 맛으로 바뀌어서요~ 더 맛있는 요리로 찾아뵐게요. 가성비 있고 푸짐한 양 담아 제공하는 팔당이 되겠습니다.<s> <b>이상, 당일 납품받는 재료로 최고의 신선도를 유지하여 드리는 회사가 되도록 노력할 것이 무엇보다 중요한데도 불구하고 저희 음식 포장 문제가 발생을 하니 정말이지에서 누락되는 경우가 발생되고 있네요, 다시 한
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 고객님들께 만족감을 드리려고 연구하고 노력하고 있는데 아직 많이 부족한 것 같습니다 시간 내시어나 말씀해 주시면 감사하겠습니다.<s> <blight difference in size of the nice wit must have big chance, I recommend you try little phortfyetangnam. HMT 4. 콜라 1.
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot >  양적으로 실망하셨다니 너무 죄송합니다. 다음번엔 더욱 맛있는 음식으로 보답 드리도록 하겠습니다.<s> 감사합니다! <s> 사진은 다시 한번 더 수정해서 올리거나

- 0906_positive_kogpt2_chat

In [ ]:
chatbot(pos_model,tokenizer,128,device)

user > 맛이 없어요
bot >  안녕하세요! 푸짐하고 맛있는 헬로 팬입니다 맛있게 잘 드셔주신 것 같아 기분 좋네요. 앞으로도 변함없는 모습으로 보답하는헬로, 닭갈비 생각나는 날 되겠습니다, 다음에 또 만나 <unk> 주세요 
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 안녕하세요! 소중한 별점 만 점 리뷰 감사합니다 즐거운 식사가 되신 거 같아 너무 뿌듯해요, 행복하게 보내셔서 진심으로 감동의 물결이 휘몰아치네요. 기분 좋은 칭찬 사진 너무나도 이쁘게 찍어주셨는데 맛있게 드시고 예쁜 포토리뷰까지 남겨 주시니 그저 아쉽지 않습니다, 이렇게 받은 신뢰에 꼭 보답할 수 있도록 매일 신선한 재료로 정성을 다하겠습니다.
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 안녕하세요. 이렇게 저희에게 귀하고 소중한 평가 남겨주셔서 너무 감사해 앞으로도 신선한 재료 아끼지 않고 정성을 가득 담아 준비 약속드릴게요, 무더운 날씨 항상 더위 조심하시고 늘 행복만 전해드리겠습니다! 
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot > 안녕하세요~ 소중한 시간 내셔서 리뷰까지 감동입니다! 고객님께서 남겨주신 별점에, 저희를 춤추게 합니, 기분이 좋습니다, 항상 힘내어 드실 수 있도록 꾸준히 노력하겠습니다. 앞으로도 행복한 하루 보내시길 바랄 테니 많이 애용해 주시고, 감기 조심하시고 파이팅 하오 
-------------------------------